In [3]:
import os

from d_2021.config.path import *
from d_2021.utils.import_utils import *
from d_2021.utils.math_utils import *
from d_2021.utils.common_utils import *

from tqdm import *
import pandas as pd

# 第一步：数据清洗

In [4]:
def data_pre_threat():
    if os.path.exists(md_df_path):
        md_df_3 = pd.read_csv(md_df_path)
        md_names = md_df_3.columns.tolist()
    else:
        md_names, md_df = get_objs_in_csv("../dataset/train/Molecular_Descriptor.csv")
        print('总特征数:', len(md_names) - 1)
        error_names = []
        # 使用箱线法去除异常值出现大于100次的列
        for md_name in md_names[1:]:
            row = md_df[md_name].tolist()
            # row = str_list_2_int(row)
            min_num, Q1, Q2, Q3, max_num = get_box_plot(row, md_name, False)
            # 计算异常值数量
            error_count = 0
            for i in range(len(row)):
                cell = md_df[md_name][i]
                if cell < min_num:
                    error_count += 1
                    md_df.loc[i, md_name] = min_num
                if cell > max_num:
                    error_count += 1
                    md_df.loc[i, md_name] = max_num
            if error_count > 100:
                error_names.append(md_name)
        print('异常值数量:', len(error_names))
        md_df_2 = remove_names(md_df, md_names, error_names)
        # 含0大于0.9的列
        zero_names = []
        # 计算列内含0大于90%的列
        for md_name in md_names[1:]:
            row = md_df_2[md_name]
            p = count_zero(row)
            if p > 0.9:
                zero_names.append(md_name)
        print('0值大于90%数量:', len(zero_names))
        md_df_3 = remove_names(md_df_2, md_names, zero_names)
        print('筛选后总数', len(md_names) - 1)
        md_df_3.to_csv(md_df_path, index=False)
    return md_df_3, md_names

In [5]:
pre_threat_df, pre_threat_names = data_pre_threat()
pre_threat_df

总特征数: 729
异常值数量: 155
0值大于90%数量: 327
筛选后总数 247


,SMILES,ALogP,AMR,apol,naAromAtom,nAromBond,nAtom,nHeavyAtom,nH,nC,...,MW,WTPT-1,WTPT-2,WTPT-3,WTPT-4,WTPT-5,WPATH,WPOL,XLogP,Zagreb
0,Oc1ccc2O[C@H]([C@H](Sc2c1)C3CCCC3)c4ccc(OCCN5C...,-0.2860,126.1188,74.170169,12.0,12.0,64.0,31.0,33.0,26.0,...,439.218115,64.771680,2.089409,15.471445,8.858910,3.406628,3011.0,47.0,4.666,166.0
1,Oc1ccc2O[C@H]([C@H](Sc2c1)C3CCCCCC3)c4ccc(OCCN...,-0.8620,131.9420,80.357341,12.0,12.0,70.0,33.0,37.0,28.0,...,467.249415,68.960024,2.089698,15.486947,8.863774,3.406648,3516.0,54.0,5.804,174.0
2,Oc1ccc(cc1)[C@H]2Sc3cc(O)ccc3O[C@H]2c4ccc(OCCN...,0.7296,139.9304,74.064997,18.0,18.0,62.0,33.0,29.0,27.0,...,463.181729,68.748923,2.083301,18.011114,11.390412,3.406644,3542.0,52.0,2.964,176.0
3,Oc1ccc2O[C@H]([C@@H](CC3CCCCC3)Sc2c1)c4ccc(OCC...,-0.3184,133.4822,80.357341,12.0,12.0,70.0,33.0,37.0,28.0,...,467.249415,68.883696,2.087385,15.468365,8.857943,3.406624,3594.0,50.0,6.015,174.0
4,Oc1ccc2O[C@H]([C@@H](Cc3ccccc3)Sc2c1)c4ccc(OCC...,1.3551,143.1903,76.356583,18.0,18.0,64.0,33.0,31.0,28.0,...,461.202465,68.883696,2.087385,15.468365,8.857943,3.406624,3594.0,50.0,4.462,174.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1969,COc1cc(OC)cc(\C=C\c2cccc(OS(=O)(=O)C3CC4OC3C(=...,1.8193,177.6817,89.159790,24.0,24.0,73.0,43.0,30.0,34.0,...,598.166139,88.709996,2.063023,25.470481,20.590936,0.000000,7121.0,70.0,2.526,236.0
1970,Oc1ccc(cc1)C2=C(C3OC2CC3S(=O)(=O)Oc4cccc(\C=C\...,1.6903,167.6057,82.972618,24.0,24.0,67.0,41.0,26.0,32.0,...,570.134839,84.662088,2.064929,24.928962,20.590936,0.000000,6171.0,66.0,1.884,228.0
1971,Oc1ccc(cc1)C2=C(C3OC2CC3S(=O)(=O)Oc4ccc(\C=C\c...,1.6903,167.6057,82.972618,24.0,24.0,67.0,41.0,26.0,32.0,...,570.134839,84.660642,2.064894,24.923083,20.590936,0.000000,6421.0,66.0,1.884,228.0
1972,Oc1ccc(cc1)C2=C([C@H]3O[C@H]2C[C@@H]3S(=O)(=O)...,1.3365,125.5605,63.287860,18.0,18.0,51.0,31.0,20.0,24.0,...,436.098059,64.171346,2.070043,19.841924,16.326873,0.000000,2583.0,50.0,0.782,174.0


# 第二步：灰色关联度分析

In [6]:
def get_sequence(m_df, names, csv_path):
    """
    得到计算好的序列
    :param m_df: 数据表
    :param names: 名称
    :param csv_path: 路径
    :return: 处理好的序列
    """
    if os.path.exists(csv_path):
        sequence = pd.read_csv(csv_path)
    else:
        sequence = get_sequence_divide_mean(m_df, names)
        sequence.to_csv(csv_path, index=False)
    return sequence

In [7]:
def get_mean_sequence():
    era_names, era_df = get_objs_in_csv("../dataset/train/ERa_activity.csv")
    # 得到母序列均值商
    parent = get_sequence(era_df, ['pIC50'], csv1_path)
    # 得到子序列均值商
    child_names1 = pre_threat_names[1:]
    child = get_sequence(pre_threat_df, child_names1, csv2_path)
    return parent, child, child_names1

## 指标正向化，确定分析数列，变量预处理

In [8]:
parent_sequence, child_sequence, child_names = get_mean_sequence()

100%|██████████| 247/247 [00:16<00:00, 15.25it/s]


In [9]:
def cal_ab():
    local_min = 100
    local_max = 0
    for name in pre_threat_names[1:]:
        # 对每一个子序列，计算对应值
        sequence = child_sequence[name]
        for i in range(len(sequence)):
            # 每一行的数相减
            t = abs(sequence[i] - parent_sequence['pIC50'][i])
            if t < local_min:
                local_min = t
            if t > local_max:
                local_max = t
    return local_min, local_max

In [10]:
a, b = cal_ab()

## 计算子序列中各个指标与母序列的关联系数

In [ ]:
p = 0.5

In [16]:
def cal_relevancy():
    copy_child = child_sequence.copy()
    if os.path.exists(csv3_path):
        copy_child = pd.read_csv(csv3_path)
    else:
        for name in tqdm(child_names):
            # 对每一个子序列，计算对应值
            sequence = copy_child[name]
            for i in range(len(sequence)):
                # 计算关联度
                sequence[i] = (a + p * b) / (abs(sequence[i] - parent_sequence['pIC50'][i]) + p * b)
        copy_child.to_csv(csv3_path, index=False)
    return copy_child

In [17]:
relevancy_df = cal_relevancy()

In [53]:
def cal_relevancy_mean(m_df: pd.DataFrame):
    obj_list = {}
    for name in tqdm(child_names):
        # 对每一个子序列，计算平均值
        m_mean = m_df[name].mean()
        obj_list[name] = m_mean
    # t_df = pd.DataFrame(obj_list)
    return obj_list

In [54]:
relevancy_mean_df = cal_relevancy_mean(relevancy_df)

100%|██████████| 247/247 [00:00<00:00, 54342.90it/s]


In [55]:
t_relevancy_mean_df = sorted(relevancy_mean_df.items(), key=lambda x: x[1], reverse=True)

获取灰色关联度前三十个

In [57]:
gray_name = []
for t in t_relevancy_mean_df:
    gray_name.append(t[0])

['BCUTp-1h',
 'MLogP',
 'VAdjMat',
 'maxaaCH',
 'ETA_Psi_1',
 'WTPT-2',
 'ETA_Epsilon_3',
 'PetitjeanNumber',
 'maxwHBa',
 'ETA_BetaP_s',
 'ETA_Epsilon_5',
 'ETA_Epsilon_2',
 'AMR',
 'SP-2',
 'Zagreb',
 'SP-3',
 'CrippenMR',
 'ETA_Alpha',
 'SP-0',
 'nC',
 'nBonds',
 'nHeavyAtom',
 'ETA_Epsilon_4',
 'WTPT-1',
 'MW',
 'SP-1',
 'ETA_Eta_R_L',
 'BCUTp-1l',
 'SP-4',
 'CrippenLogP',
 'SP-5',
 'WPOL',
 'ETA_EtaP_L',
 'VP-0',
 'minaaCH',
 'MLFER_L',
 'McGowan_Volume',
 'ATSm3',
 'VABC',
 'ETA_EtaP',
 'sumI',
 'ETA_Epsilon_1',
 'ETA_Beta',
 'apol',
 'ATSm2',
 'ATSm4',
 'ETA_Beta_s',
 'VP-1',
 'Kier1',
 'MLFER_E',
 'ATSp1',
 'minHother',
 'ATSp2',
 'ETA_BetaP',
 'VP-2',
 'nRing',
 'minHaaCH',
 'maxHother',
 'maxHaaCH',
 'SP-6',
 'ETA_Eta_B_RC',
 'nBonds2',
 'nwHBa',
 'nAtom',
 'gmax',
 'maxHBa',
 'ATSp3',
 'SPC-4',
 'ATSm5',
 'ETA_Shape_Y',
 'ETA_Beta_ns',
 'MDEC-23',
 'FMF',
 'ETA_Eta_F_L',
 'naAromAtom',
 'VP-3',
 'ATSp4',
 'nBondsD',
 'ETA_Eta_R',
 'C2SP2',
 'n6Ring',
 'nBondsM',
 'ETA_EtaP_F

In [ ]:
from scipy import *
